## Collab setup

In [ ]:
!pip install pycbc lalsuite bilby nestle gwpopulation matplotlib

## PC setup

### Using anaconda/miniconda
`conda env create -f environment.yaml`

`conda activate np3m`

OR

### Using pip
Create a python virtual environment and 
`pip install -r requirements.txt`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import dynesty
from dynesty import DynamicNestedSampler

# Define the model

In [ ]:
def linear_model(x, parameters):

    m, c = parameters
    return m*x + c


def quadratic_model(x, parameters):

    a, b, c = parameters
    return a*x**2 + b*x + c



# Create mock data

In [ ]:
Npoints = 1000

x = np.linspace(1, 4, Npoints)

sigma = 6

noise = sigma*np.random.randn(Npoints)
m0, c0 = 5, -4

l_truths  = np.array([m0, c0])

signal = linear_model(x, l_truths)

data = signal + noise


plt.scatter(x, data, s=1)
plt.plot(x, signal, c='red')
plt.show()

# Define likelihood and prior

In [ ]:
class SimpleLikelihoodModel:
    def __init__(self, x, data, model):
        self.x = x
        self.data = data
        self.model  = model


    def loglikelihood(self, parameters):

        prediction = self.model(self.x, parameters)

        return -0.5* np.sum ( ((self.data - prediction)/sigma)**2)
    

# Setup sampler

In [ ]:
def l_priors(parameters_t):

    m_t, c_t = parameters_t

    m = 10*m_t
    c = -10 + 20*c_t

    return np.array([m, c])


slm = SimpleLikelihoodModel(x=x,
                            data=data,
                            model=linear_model)

l_sampler = DynamicNestedSampler(loglikelihood=slm.loglikelihood,
                               prior_transform=l_priors,
                               nlive=100,
                               ndim=2)



In [ ]:

l_sampler.run_nested()

In [ ]:
from dynesty.plotting import cornerplot, traceplot, runplot

In [ ]:
runplot(l_sampler.results)

In [ ]:
traceplot(l_sampler.results, truths=l_truths)

In [ ]:
cornerplot(l_sampler.results, truths=l_truths)

In [ ]:
n1 = int(0.68*len(l_sampler.results.samples))

samples = l_sampler.results.samples
logl = l_sampler.results.logl
order = np.argsort(logl)[::-1]

rlogl = logl[order]
rsamples = samples[order]


fsamples = []
for idx in range(n1):
    fsamples.append(rsamples[idx])

fsamples = np.array(fsamples)

In [ ]:
fsamples[:, 0]

In [ ]:
fig, ax = plt.subplots(figsize=(18, 12))


for sample in fsamples:
    ax.plot(x, linear_model(x, sample), c='cyan', alpha=0.1)

ax.grid()
ax.plot(x, linear_model(x, l_truths), c='magenta')
plt.show()

In [ ]:
ev_l = l_sampler.results.logz[-1]

## Quadratic model

In [ ]:
def q_priors(parameters_t):

    a_t, b_t, c_t = parameters_t

    a = -10 + 20*a_t
    b = -10 + 20*b_t
    c = -10 + 20*c_t

    return np.array([a, b, c])


sqm = SimpleLikelihoodModel(x=x,
                            data=data,
                            model=quadratic_model)

q_sampler = DynamicNestedSampler(loglikelihood=sqm.loglikelihood,
                               prior_transform=q_priors,
                               nlive=100,
                               ndim=3)


In [ ]:

q_sampler.run_nested()

In [ ]:
traceplot(q_sampler.results, truths=[0, m0, c0])

In [ ]:
cornerplot(q_sampler.results, truths=[0, m0, c0])

In [ ]:
ev_q = q_sampler.results.logz[-1]

In [ ]:
logB = ev_l - ev_q

In [ ]:
B = 10**(logB)

print(B)